# <center> Formation à la manipulation de données textuelles en Python </center>
## <center>  Jean-Philippe Magué (ENS de Lyon) <br/> Alioscha Massein (Maison des Sciences de l'Homme - Lyon Saint Etienne) <br/> Sylvain Besson (Maison des Sciences de l'Homme - Lyon Saint Etienne)</center> 

# 0. Préambule
## 0.1 Les notebooks Jupyter
Ceci est un *[notebook Jupyter](https://jupyter-notebook.readthedocs.io/en/stable/index.html)*. C'est un document, ou plus précisément une application web, permettant d'exécuter du code Python dans un navigateur web. Les notebooks présentent de nombreux intérêts : interactivité, possiblité de mélanger codes et textes (et images), possibilité d'exécuter le code sur une machine distante...

Un notebook est une succession de *cellules*. Il y a différents types de cellules, notamment texte (et même [Markdown](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html)) et code. 

Vous êtes en train de lire une cellule Markdown : si vous double-cliquez, vous pourrez l'éditer.

La cellule suivante est une cellule de code : si vous tapez du code dedans, vous pourrez l'exécuter. 

In [26]:
2+2

4

L'exécution d'une cellule de code affiche toujours le résultat de la dernière instruction. 

## 0.2 Programme de la journée
![](images/img1.png)

# 1. Récupération de données en ligne
Nous allons récupérer 2 types d'information sur le site de la MSH :
* La [liste](https://www.msh-lse.fr/laboratoires/) des tous les laboratoires, avec pour chacun son nom, son acronnyme, son code, ses disciplines et l'adresse de la page le décrivant
* Pour chaque laboratoire, le *Projet scientifique* et les *Compétences, activités valorisables*

Pour cela, nous allons nous appuyer sur 2 packages Python : [requests](https://requests.readthedocs.io/en/latest/) qui permet, entre autres, de faire des requêtes HTTP et [Beautiful Soup](https://beautiful-soup-4.readthedocs.io/en/latest/) qui permet de parser et d'extraire des parties de documents HTML (et XML)

In [27]:
import requests
from bs4 import BeautifulSoup

## Exemple

In [28]:
url='http://perso.ens-lyon.fr/jean-philippe.mague/other/cours/2021-2022/IXXI/manipText/exemple.html' 
html=requests.get(url).text
#c'est le document html utilisé comme exemple dans la documentation de Beautiful Soup

In [29]:
print(html)#html est une chaîne de caractères

<!DOCTYPE html>
<html lang="fr" >
    <head>
        <title>The Dormouse's story</title>
    </head>
    <body>
        <p class="title"><b>The Dormouse's story</b></p>

        <p class="story">Once upon a time there were three little sisters; and their names were
        <a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
        <a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
        <a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
        and they lived at the bottom of a well.</p>

        <p class="story">...</p>
    </body>
</html>



In [30]:
soup = BeautifulSoup(html, 'html.parser')

In [31]:
print(soup) #soup est un objet complexe qui permet de naviguer dans l'arbre HTML

<!DOCTYPE html>

<html lang="fr">
<head>
<title>The Dormouse's story</title>
</head>
<body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
        <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
        <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
        <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
        and they lived at the bottom of a well.</p>
<p class="story">...</p>
</body>
</html>



In [32]:
soup.title #le premier élément <title>

<title>The Dormouse's story</title>

In [33]:
soup.title.string

"The Dormouse's story"

In [34]:
soup.a #le premier élément <a>

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [35]:
print(soup.a['class'])# On peut accéder aux attributs d'un élément
print(soup.a['href'])

['sister']
http://example.com/elsie


In [36]:
soup.find_all('a') #on peut rechercher tous les éléments à partir de leur nom

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [37]:
soup.find_all('p',{"class": "story"}) #on peut également imposer des contraintes sur leurs attributs

[<p class="story">Once upon a time there were three little sisters; and their names were
         <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
         <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
         <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
         and they lived at the bottom of a well.</p>,
 <p class="story">...</p>]

## Liste des laboratoires

La liste des laboratoires est disponible [ici](https://www.msh-lse.fr/laboratoires/). 
### Exercice 1.1
Comment récupérer le code HTML de la page ?

In [38]:
url='https://www.msh-lse.fr/laboratoires/'
html=requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

La structure de la page est la suivante :

![](images/img2.png)

### Exercice 1.2
Comment récupérer les cartes qui représentent chaque laboratoire ?

In [39]:
cards=soup.find_all('div',{"class": "card-project-lab"})

### Exercice 1.3
Etant donnée une carte représentant un laboratoire, comment récupérer son nom, son acronyme, son code, ses disciplines et l'adresse de la page le décrivant ?

In [40]:
card = cards[0]
print(card.a['href'])
print(card['data-disciplines'])
print(card.h3.text)
print(card.div.text)
print(card.p.text)

https://www.msh-lse.fr/laboratoires/arar/
archeology,economy,history
ARAR
Archéologie et Archéométrie
UMR 5138


### Exercice 1.4

On va représenter l'ensemble des informations sur tous les labos comme un dictionnaire de dictionnaires : 

```python
{
  "ARAR": {
    "nom": "Archéologie et Archéométrie",
    "code": "UMR 5138",
    "disciplines": "archeology,economy,history",
    "url": "https://www.msh-lse.fr/laboratoires/arar/"
  },
  "ARCHEORIENT": {
    "nom": "Environnements et sociétés de l'Orient ancien",
    "code": "UMR 5133",
    "disciplines": "archeology,geography,history",
    "url": "https://www.msh-lse.fr/laboratoires/archeorient/"
  },

```

In [41]:
labos={}
for card in cards:
    sigle=card.h3.text
    labo={}
    labo['nom']=card.div.text
    labo['code']=card.p.text
    labo['disciplines']=card['data-disciplines']
    labo['url']=card.a['href']
    labos[sigle]=labo

### Sauvegarde des données

C'est le bon moment pour enregistrer les données que nous venons de récupérer et de structurer. Le format *json* est particulièrement bien adapté.

Sous Windows, si l'on souhaite que le fichier soit encodé en Unicode (ce qui est hautement conseillé), on est obligé de préciser explicitement. Sous Mac et Linux, c'est l'encodage par défaut. 

In [42]:
import json

In [43]:
with open('labos.json', 'w', encoding='utf8') as f:
    f.write(json.dumps(labos))
    #f.write(json.dumps(labos, indent=4)) #On peut préférer cette version si l'on souhaite que le fichier soit lisiblement formaté.

## Textes de chaque labo

In [44]:
# Si besoin, on peut recharger les données
with open('labos.json', encoding='utf8') as f:
    labos = json.loads(f.read())

Le principe pour aller récupérer le projet scientifique et les activités valorisables de chaque labo est le même que ci dessus : on récupère le code HTML disponible à l'URL de la page de description de chaque labo, on parse ce code HTML avec Beautiful Soup et on va chercher les informations pertinentes. 

### Exercice 1.5
Compléter la cellule ci-dessous

In [45]:
from tqdm.notebook import tqdm #tqdm est bibliothèque qui permet d'avoir une barre de progression 

projets={}
compétences={}
for labo in tqdm(labos):
    html=requests.get(labos[labo]['url']).text
    soup = BeautifulSoup(html, 'html.parser')
    try:
        h2_projet=soup.find("h2", string="Projet scientifique")
        projets[labo]=h2_projet.find_next_sibling('div').text
    except Exception as e:
        print(f"Impossible de récupérer le projet scientifique du laboratoire {labo} : {e}")
    try:    
        h2_compétences=soup.find("h2", string="Compétences, activités valorisables")
        compétences[labo]=h2_compétences.find_next_sibling('div').text
    except Exception as e:
        print(f"Impossible de récupérer les compétences du laboratoire {labo} : {e}")

  0%|          | 0/53 [00:00<?, ?it/s]

Impossible de récupérer le projet scientifique du laboratoire CERCRID : 'NoneType' object has no attribute 'find_next_sibling'
Impossible de récupérer le projet scientifique du laboratoire CLHDPP : 'NoneType' object has no attribute 'find_next_sibling'
Impossible de récupérer les compétences du laboratoire CLHDPP : 'NoneType' object has no attribute 'find_next_sibling'
Impossible de récupérer les compétences du laboratoire ECLLA : 'NoneType' object has no attribute 'find_next_sibling'
Impossible de récupérer les compétences du laboratoire FMRI : 'NoneType' object has no attribute 'find_next_sibling'
Impossible de récupérer les compétences du laboratoire GREPS : 'NoneType' object has no attribute 'find_next_sibling'
Impossible de récupérer les compétences du laboratoire IETT : 'NoneType' object has no attribute 'find_next_sibling'
Impossible de récupérer les compétences du laboratoire IRPHIL : 'NoneType' object has no attribute 'find_next_sibling'
Impossible de récupérer les compétences

### Sauvegarde des données
On veut enregistrer les données que l'on vient de récupérer. On souhaite la structure de fichiers suivante : 
```
.
├── labos.json
├── labos/
│   ├──ARAR/    
│   │  ├── projet_scientifique.txt
│   │  ├── Compétences_activités_valorisables.txt
│   ├──ARCHEORIENT/   
│   │  ├── projet_scientifique.txt
│   │  ├── Compétences_activités_valorisables.txt
...
```
Python crée automatiquement les fichiers inexistants lorsqu'on les ouvre (en mode écriture), autant il ne crée par les dossiers : il faut le faire explicitement. Le package [pathlib](https://docs.python.org/3/library/pathlib.html) permet ce genre de manipulation.

In [46]:
from pathlib import Path

In [48]:
Path('labos').mkdir(exist_ok=True)
for labo in projets:
    Path(f'labos/{labo}').mkdir(exist_ok=True)
    with open(f'labos/{labo}/projet_scientifique.txt', 'w', encoding='utf8') as f:
        f.write(projets[labo])
for labo in compétences:
    Path(f'labos/{labo}').mkdir(exist_ok=True)
    with open(f'labos/{labo}/Compétences_activités_valorisables.txt', 'w', encoding='utf8') as f:
        f.write(compétences[labo])        